# Coursework 2 - Atomistic simulation

Please enter your name and student number into the Markdown cell below.

#### Name:
#### Student Number:

In this coursework we will explore some important concepts from atomistic simulation in more detail. To keep the simulations simple, we will be simulating binary collisions rather than collision cascades, which can take a long time to run.

## Setup 

Please enter your Student ID in the cell below. This ID will be used throughout the coursework so please do not create any other variables named `ID`.

In [1]:
ID = 7023839
vpka = [50.0 + 10.0*ID%14747/14746,0.0,0.0]
print('Your PKA velocity for this notebook is defined as: (%f,%f,%f) ang/ps' % (vpka[0],vpka[1],vpka[2]))

Your PKA velocity for this notebook is defined as: (50.893530,0.000000,0.000000) ang/ps


Run the below cell to import some key modules.

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import timeit
from scipy.optimize import curve_fit 

We need to define all the functions needed to run a small molecular dynamics simulation. The cell below gathers together all the required functions from the Atomistic Simulation notebook. Execute the cell to define the functions: 

In [3]:
# Calculate Lennard-Jones potential energy of a pair of atoms
def lj(R,eps,r0,rcut,alpha):
    if R<=rcut:
        return eps*((r0/R)**12-2.0*(r0/R)**6)+alpha*eps
    else:
        return 0.0
    
# Calculate Lennard-Jones force between a pair of atoms
def flj(R,eps,r0,rcut,alpha):
    if R<=rcut:
        return 12*eps/r0*((r0/R)**7-(r0/R)**13)
    else:
        return 0.0
# Return potential energy of a system of atoms
def pe(r):
    pe = 0.0
    for i in range(len(r)):
        for j in range(i):
            disp = np.abs(r[j,:]-r[i,:]) # Calculate displacement between atoms
            for s in range(3): # Check if atoms are closer across periodic boundaries in each dimension
                if disp[s] > 0.5*boxsize[s]*cell[s,s]:
                    disp[s] = disp[s] - boxsize[s]*cell[s,s] # Adjust displacement for periodic boundary  
            R = np.linalg.norm(disp)
            pe = pe + lj(R,eps,r0,rcut,alpha)
    return pe

# Return kinetic energy of a system of atoms
def ke(v,m):
    mconv = 1.036e-4
    ke = 0.0
    for i in range(len(r)):
        ke = ke + m[i]*(v[i,0]*v[i,0] + v[i,1]*v[i,1] + v[i,2]*v[i,2])
    return 0.5*mconv*ke

# Calculate acceleration of a system of atoms
def calcaccln(a,r,m):
    mconv = 1.036e-4
    for i in range(len(r)):
        a[i,:] = 0.0
    for i in range(len(r)):
        for j in range(i):
            disp = (r[j,:]-r[i,:]) # Calculate displacement between atoms
            for s in range(3): # Check if atoms are closer across periodic boundaries in each dimension
                if disp[s] > 0.5*boxsize[s]*cell[s,s]:
                    disp[s] = disp[s] - boxsize[s]*cell[s,s] # Adjust displacement for periodic boundary
                elif disp[s] < -0.5*boxsize[s]*cell[s,s]:
                    disp[s] = disp[s] + boxsize[s]*cell[s,s] # Adjust displacement for periodic boundary    
            R = np.linalg.norm(disp)
            a[i,:] = a[i,:] + flj(R,eps,r0,rcut,alpha)*disp[:]/R/m[i]/mconv
            a[j,:] = a[j,:] - flj(R,eps,r0,rcut,alpha)*disp[:]/R/m[j]/mconv
    return R

# Evolve the positions, velocities and accelerations of a system of atoms by one timestep
def vv(r,v,a,m,dt):
    v += 0.5*a*dt
    r += v*dt
    calcaccln(a,r,m)
    v += 0.5*a*dt
    for s in range(3):
        for i in range(len(r)):
            if r[i,s] > boxsize[s]*cell[s,s]:
                r[i,s] = r[i,s] - boxsize[s]*cell[s,s]
    return

# Run a simulation and record key data
def run(r,v,a,m,dt,nsteps):
    R0 = [r[0,:].tolist()]
    R1 = [r[1,:].tolist()]
    V0 = [v[0,:].tolist()]
    V1 = [v[1,:].tolist()]
    A0 = [a[0,:].tolist()]
    A1 = [a[1,:].tolist()]
    pelog = [pe(r)]
    kelog = [ke(v,m)]
    #print('Running simulation of ' + str(nsteps) + ' steps')
    #print('Progress:')
    for t in range(nsteps):
        #if (t+1)%10 == 0:
        #    print(str(t+1) + '.', end='')
        vv(r,v,a,m,dt)
        R0.append(r[0,:].tolist())
        R1.append(r[1,:].tolist())
        V0.append(v[0,:].tolist())
        V1.append(v[1,:].tolist())
        A0.append(a[0,:].tolist())
        A1.append(a[1,:].tolist())
        pelog.append(pe(r))
        kelog.append(ke(v,m))
        #if (t+1)%100 == 0:
        #    print('')
    #print()
    return np.array(R0),np.array(R1),np.array(V0),np.array(V1),np.array(A0),np.array(A1),np.array(pelog),np.array(kelog)

### A note on units 
I have set up the Lennard-Jones potential and the initial velocity so that times are specified in picosecond (ps), distances in angstrom and energies in electron-volts (eV). Please use this system on units in your answers.

## Part 1 - The importance of the choice of timestep (13 marks)
Now we have everything set up. In this part of the coursework we will explore how the choice of timestep affects the behaviour and reliability of a simulation.

#### <span style="color: red"> Task 1a:</span> Set up a simulation cell for a binary collision (3 marks)

For guidance, look at the example of a binary collision at the end of the class notebook. Set up a simulation box of $8\times 4\times 4$ unit cells with a lattice parameter of 2.86 ang. You can use the motif for a bcc lattice, even though we are not actually going to populate the box with a full crystal. define an array `r[]` which places atoms at $(2.0,5.0,5.0)$ and $(7.0,5.0,5.0)$. Initialise the velocity and acceleration arrays with zeros and the mass array with all atoms having the Fe mass 55.8. Set up the parameters for the Lennard-Jones potential as in class.

#### <span style="color: red"> Task 1b:</span> Visualise the simulation cell (1 mark)
Produce a simple plot of the positions of the atoms as points in the x-y plane. Include some indication of the boundaries of the simulation cell, either as lines in the plot or by adjusting the x and y ranges of the plot.

#### <span style="color: red"> Task 1c:</span> Run a simulation (1 mark)
Initialise the velocity of atom 0 in the simulation to the value of vpka that was initialised at the start of the notebook. Run a simulation with a timestep of 0.0001 for 2000 timesteps. 

#### <span style="color: red"> Task 1d:</span> Plot the change in total energy as a function of time (2 marks)
Produce a plot of the change in total energy (i.e. the difference between the total energy at each timestep and the energy at the start of the simulation) against the number of timesteps into the simulation. You should see two step-like features and an oscillating feature. In a markdown cell, explain what gives rise to each of these features [Hint: it might help to think about what this plot would look like if the simulation was perfect].

#### <span style="color: red"> Task 1e:</span> Explore the effect of the timestep on energy conservation ( 2 marks)
Run simulations of collisions just like the one above for a range of values of the timestep `dt`. Use the following values for the timestep $[0.00001, 0.0001, 0.001, 0.01]$. For each choice of timestep you will need a different number of steps in your simulation to ensure that the overall duration of the simulation remains the same. The corresponding set of numbers of steps will be $[20000,2000,200,20]$.

For each simulation, record the change in total energy between the first and last step in a list. This is a measure of the lack of energy conservation in the simulation.

[Hint: By placing the values for the timestep and the number of steps in a list, you can define a loop and for each simulation append the change in total energy to a list of energy changes. This is the most efficient way to achieve the task and you will  gain full marks only if you use this approach. An alternative is to explore the timesteps manually and record the resulting energy differences in a list by hand, but this will not gain you full marks. The class notebook included an example of using a python loop to execute a series of simulations.]

[Hint: Remember that for each simulation you will need to reinitialise the arrays of positions, velocities and acceleration, otherwise you will start with the values from the end of the previous simulation!] 

#### <span style="color: red"> Task 1f:</span> Plot a graph of energy change against the value of the timestep `dt` (2 marks)
Plot the absolute value of the energy change against the value of timestep `dt` using a log-log scale. Remember that the total energy should be conserved, so any change represents an error in our simulation.

In a markdown cell, comment on the resulting graph. How would you explain the resulting behaviour? At what value of the timestep does the change in total energy become intolerably large?

#### <span style="color: red"> Task 1g:</span> Plot a graph of the distance moved by the projectile atom (atom 0) in the first timestep of the simulation (2 marks)
Calculate the distance moved by the projectile atom for each timestep and plot this against the timestep on a log-log plot (Hint: you do not need to any simulation results in order to do this). In a markdown cell, comment on the value that you obtain for the displacement per timestep for the longest timestep in light of your plot of energy change above. 

## Part 2 - Energy transfer (8 marks)
We will now investigate the amount of energy tranferred from the projectile atom (atom 0) to the target atom (atom 1) during the collision as a function of the *impact parameter* of the collision. The impact parameter is defined as the perpendicular distance of the target atom from a straight line projection of the initial trajectory of the projectile atom as shown below

![](Figures/ImpactParameter.png "")


As well as simulating the collision process we will consider it in an analytical treatment and see how well a simple theory matches our results.

#### <span style="color: red"> Task 2a:</span> Define a function to return the value of the kinetic energy for a single atom (1 mark)
In the code cell below, define a new function called `keatom()` which takes two arguments, the first being an array (or list) containing the three components of an atom's velocity and the second being the mass of the atom. The function should return the kinetic energy of the atom (in eV).

[Hint: The function `ke()` defined at the start of the notebook will provide a useful guide for how to complete this task.]

#### <span style="color: red"> Task 2b:</span> Run simulations with a variety of impact parameters (2 marks)
Run a set of six simulations with impact parameters `b = [0.0,0.1,0.2,0.5,1.0,2.0]` and for each simulation record the fraction of the initial projectile kinetic energy transferred to the target atom. For each simulation, use the initial velocity `vpka` that was set at the start of the notebook. Once again, to gain full marks you must use a loop to run the multiple simulations. A timestep of 1e-4 should be suitable.

[Hint: To give the correct impact parameter, simply set the starting position of the projectile atom for each value of `b` to `[2.0,5.0+b,5.0]`.

[Hint 2: Remember that the kinetic energy of the target atom is zero at the start of the simulation. The potential energy will be zero at the start and end because the atoms are out of range of one another. This makes it easy to calculate the kinetic energy gained by the target.] 

#### <span style="color: red"> Task 2c:</span> Plot the fraction of kinetic energy trasferred as a function of impact parameter (1 mark)
Plot the results of the above computer experiment on a plot with linear scales.

### A theoretical treatment of the collision
The above plot should show a well behaved trend. Can we explain the form of this trend using some simple theory?

First, let's make some simplifications in our analysis of the collision process. Let's assume that the interaction between the atoms is only significant when they are within a certain range that we denote $r^{\ast}$. This means that the forces between the atoms will only act to transfer energy when the atoms are within a distance of $r^{\ast}$ or less. We will also assume that we can approximate the energy transfer in the true collision process by assuming that it is the same as would have occurred if the particles followed their unperturbed paths (i.e. the target atom remains stationary and the projectile moves in a straight line. The figure below illustrates this approximate scenario for a given impact parameter.

![](Figures/CollisionTime.png "Logo Title Text 1")


You may recall that the change in momentum of a particle (also called an *impulse*) $\Delta p$ of a particle experiencing a force $F$ for a time $t$ is given by $\Delta p = F t$ (you can see this if you remember Newton's second law, that a force is a rate of change of velocity multipled by mass). Now, since the target particle is initially at rest, the kinetic energy transfer is simply the value of its final kinetic energy. If it receives momentum $\Delta p$ then this kinetic energy change is:

$$
\Delta E_{\mathrm{kin}} = \frac{\Delta p^2}{2m}.
$$

Now, from the figure, we can see that the length of the projectile trajectory within range $r^{\ast}$ is given by $x = 2\,\sqrt{{r^{\ast}}^2-b^2}$ which, for a projectile speed $v$ implies an interaction time 

$$
t = x/v = 2(\sqrt{{r^{\ast}}^2-b^2})/v.
$$

Inserting this into our expression for the kinetic energy transfer and gathering together the various factors into a constant $\beta$ gives

$$
\Delta E_{\mathrm{kin}} = \frac{\Delta p^2}{2m} = \frac{(F\,t)^2}{2m} = \frac{F^2}{2m v^2}4({r^{\ast}}^2-b^2) = \beta \,({r^{\ast}}^2-b^2).
$$

Now, we know from basic physics that in a perfectly elastic head-on collision between two particles of equal mass, the kinetic energy transfer is complete. This is to say that the value of $\Delta E_{\mathrm{kin}}$ must be 1.0 when $b=0$, which means we must have $\beta { r^{\ast}}^2=1$,  and so our final theoretical expression reads:

$$
\Delta E_{\mathrm{kin}} = 1 -\beta b^2.
$$

A more sophisticated treatment would allow us to calculate the value of the parameter $\beta$ directly, but this would involve an integral over the Lennard-Jones potential and would in any case give the wrong answer because some of the assumptions that we have made in our analysis are a little too agressive for the low collision velocities that we are simulating. However, what we can do is fit the value of $\beta$ to our simulation data to verify the *form* of the model that we have derived.

#### <span style="color: red"> Task 2d:</span> Define a python function for the kinetic energy tranfer as a function of impact parameter and fit the value of the $\beta$ parameter (2 marks)
Define a python function `demodel()` that takes a value of the impact parameter `b` and a value for the constant `beta` and returns a value for the kinetic energy transfer according to the equation that we derived above. Use what you learned about curve fitting earlier in the module to fit the value of the parameter `beta` to the output from your simulations.

#### <span style="color: red"> Task 2e:</span> Plot the fitted model against the simulation data (1 mark)
Produce a plot of the simulation data and the fitted model on the same axes and comment on the fit.

#### <span style="color: red"> Task 2f:</span> Calculate the value of $r^{\ast}$ (1 mark)
Calculate the value of $r^{\ast}$ implied by your simulation data. Comment on the value you obtain with respect to the shape of the Lennard-Jones potential with the parameterisation for Fe. [Hint: Think about the shape of the Lennard-Jones curve and how it is made up of a repulsive and an attractive contribution.]

## Part 3 -  (4 marks)
Read the following instructions very carefully. To earn all the marks for this part you must do everything asked for.

#### <span style="color: red"> Task 3:</span> (4 marks)
Run a set of simulations using values for the impact parameter of `b=[0.05,0.1,0.2,0.5,1.0,2.0,3.0]`. Note that this is a slightly different set to before. Produce a *single* plot showing the trajectories followed by the projectile in the x-y plane for all of the simulations. Make sure your plot makes it possible to distinguish which trajectory corresponds to which value of the impact parameter. Next produce a plot which shows how the angle of deflection of the final projectile path varies as a function of impact parameter across your set of simulations. Finally, in a markdown cell, explain the behaviour of the simulation for `b=3.0`.

### For use by markers only:

#### Total mark: $\hspace{1cm}$/25

#### As a percentage: